In [1]:
!pip install sacremoses
from transformers import pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 21.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=6c87999e3df3e216557f6d01108ce64e5f74f95f4d889c18ced25216e11a1685
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [2]:
from transformers import BioGptForSequenceClassification, BioGptTokenizer

model_name = "microsoft/biogpt"
model = BioGptForSequenceClassification.from_pretrained(model_name, num_labels=3, problem_type="multi_label_classification")
tokenizer = BioGptTokenizer.from_pretrained(model_name)

from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="microsoft/biogpt", tokenizer="microsoft/biogpt")

Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import torch

In [4]:
def get_decision(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_class_idx = outputs.logits.argmax(dim=-1).item()
    
    return predicted_class_idx

In [5]:
import pandas as pd
summaries_df = pd.read_csv('/kaggle/input/summariespubmedabstracts/summaries.csv')
print(summaries_df.head())

         ID                                            Summary
0  30297319  Gestational diabetes mellitus (GDM) is the mos...
1  29669543  Cardiovascular disease (CVD) is the leading ca...
2  27219496  Objective: Development of these guidelines is ...
3  32899917  Background: Although the body of evidence indi...
4  33652942  Non-alcoholic fatty liver disease (NAFLD) is t...


In [6]:
MAX_TOKENS = 512

def truncate_text_to_fit(text, max_tokens=MAX_TOKENS):
    token_count = len(tokenizer.tokenize(text))
    if token_count > max_tokens - 2:  
        truncated_text = " ".join(text.split()[:-1])
        while len(tokenizer.tokenize(truncated_text)) > max_tokens - 2:
            truncated_text = " ".join(truncated_text.split()[:-1])
    else:
        truncated_text = text
        
    return truncated_text

In [7]:
sample_question= "DPP-4 inhibitor or GLP-1 receptor agonist for glycemic control and consider adding a second antihypertensive such as a calcium channel blocker or diuretic" 

In [8]:
def create_input_text(question, context):
    input_text = "question: "+ question + " context: " + context
    return input_text

summaries_df['input_text'] = summaries_df['Summary'].apply(lambda x: create_input_text(sample_question, x))
summaries_df.head()

,ID,Summary,input_text
0,30297319,Gestational diabetes mellitus (GDM) is the mos...,question: DPP-4 inhibitor or GLP-1 receptor ag...
1,29669543,Cardiovascular disease (CVD) is the leading ca...,question: DPP-4 inhibitor or GLP-1 receptor ag...
2,27219496,Objective: Development of these guidelines is ...,question: DPP-4 inhibitor or GLP-1 receptor ag...
3,32899917,Background: Although the body of evidence indi...,question: DPP-4 inhibitor or GLP-1 receptor ag...
4,33652942,Non-alcoholic fatty liver disease (NAFLD) is t...,question: DPP-4 inhibitor or GLP-1 receptor ag...


In [9]:
def save_to_csv(indexes, pubids, results, filename='results.csv'):
    df = pd.DataFrame({
        'Index': indexes,
        'PubID': pubids,
        'Result': results
    })
    if not pd.io.common.file_exists(filename):
        df.to_csv(filename, index=False)
    else:
        df.to_csv(filename, mode='a', header=False, index=False)

results = []
pubids = []
indexes = []

for idx, row in summaries_df.iterrows():
    indexes.append(idx + 1)
    pubids.append(row['ID'])
    results.append(classifier(truncate_text_to_fit(row['input_text'])))

if results:
    save_to_csv(indexes, pubids, results)

In [16]:
decisions = pd.read_csv('/kaggle/working/results.csv')
print(decisions.head())

   Index     PubID                                             Result
0      1  30297319  [{'label': 'LABEL_1', 'score': 0.7801956534385...
1      2  29669543  [{'label': 'LABEL_1', 'score': 0.7972989082336...
2      3  27219496  [{'label': 'LABEL_1', 'score': 0.7269024252891...
3      4  32899917  [{'label': 'LABEL_1', 'score': 0.7557678222656...
4      5  33652942  [{'label': 'LABEL_1', 'score': 0.8152899742126...


In [17]:
import pandas as pd
import json

label_mapping = {
    "LABEL_0": "yes",
    "LABEL_1": "no",
    "LABEL_2": "maybe"
}

def map_label(result_str):
    try:
        result_str = result_str.replace("'", '"')
        result = json.loads(result_str)
        label = result[0]['label']
        return label_mapping.get(label, "Unknown")
    except:
        return result_str

decisions['Result'] = decisions['Result'].apply(map_label)

print(decisions)


   Index     PubID Result
0      1  30297319     no
1      2  29669543     no
2      3  27219496     no
3      4  32899917     no
4      5  33652942     no
5      6  29122390     no
6      7  30179134     no
7      8  29704920     no
8      9  32529512    yes
9     10  20301444     no
